# Prétraitement des images 

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import os
from os import listdir
from sklearn import preprocessing
import cv2
import pandas as pd
import matplotlib.pyplot as plt
import time
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from keras.applications import VGG16
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from keras.models import Model

## Import du dataset

In [ ]:
path = "./Images_redim/"
list_photos = [file for file in listdir(path)]
print(len(list_photos))
list_photos[0]

In [ ]:
# Chargement du dataframe avec l'id du produit et sa catégorie
df = pd.read_csv('cleaned.csv')
df.head(2)

In [ ]:
# définir une fonction lambda pour extraire la catégorie principale
extract_category = lambda x: x.split(">>")[0].strip().strip(']["')

# appliquer la fonction à la colonne 'product_category_tree' du DataFrame
df["label_name"] = df['product_category_tree'].apply(extract_category)
df["label_name"][0]

In [ ]:
label_encoder = preprocessing.LabelEncoder()
label_encoder .fit(df["label_name"])
df['category']=label_encoder.transform(df["label_name"])
df['category'][0:3]

In [ ]:
df['filename'] = df.uniq_id + '.jpg'
df.head(3)

In [ ]:
df.columns

In [ ]:
df_image = df[['filename', 'category', 'label_name']].copy()
df_image

In [ ]:
sns.countplot(x='category',data=df_image).set_title("Data Distribution")

In [ ]:
df_image["file_path"] = path + df_image["filename"]
df_image

In [ ]:
import pandas as pd
import numpy as np
from keras.applications import VGG16
from keras.preprocessing.image import ImageDataGenerator

# Chargement du modèle pré-entrainé VGG16
vgg16_model = VGG16(weights='imagenet', include_top=False, input_shape=(800, 800, 3))

# Définition de la taille du mini-batch
batch_size = 32

# Préparation du générateur d'images
datagen = ImageDataGenerator(rescale=1./255)


# Définition du générateur de features
generator = datagen.flow_from_dataframe(
    df_image,
    directory=path,
    x_col='filename',
    y_col='label_name',
    batch_size=batch_size,
    shuffle=False,
    class_mode="categorical",
    target_size=(800, 800)
)

# Extraction des features pour chaque mini-batch
features = vgg16_model.predict(generator, verbose=1)

# Redimensionnement des features et ajout au dataframe
features = features.reshape((features.shape[0], -1))
df_features = pd.DataFrame(features)
df_features.columns = ['feature_' + str(i) for i in range(df_features.shape[1])]
df_with_features = pd.concat([df_image, df_features], axis=1)

# Enregistrement des données avec les features
df_with_features.to_csv('data_with_features.csv', index=False)